In [52]:
import pandas as pd

df = pd.read_excel('limpaBrasil.xlsx')

#### Padronizando valor de CEP sem '-'

In [105]:
# df['CEP'] = df['CEP'].str.replace(r'(\d{5})(\d{3})', r'\1-\2', regex=True) Esse é pra padronizar COM traçinho
df['Cep'] = df['Cep'].str.replace(r'\D', '', regex=True)

AttributeError: Can only use .str accessor with string values!

#### Apagando valores nulos e diferentes de 8 caracteres da Coluna CEP

In [81]:
df = df.dropna(subset=['Cep'])

In [91]:
df = df[df['Cep'].str.len() == 8]

#### Padronizando como número

In [56]:
# df['CEP_ARRUMADO'] = df['CEP_ARRUMADO'].astype(int) # Não fazer porque retira o zero na frente do CEP, dificultando achar as informações referentes a ele

#### Verificando em qual zona de SP está

In [92]:
print(df['Cep'])

Series([], Name: Cep, dtype: object)


#### Eliminando CEPs não existentes

In [58]:
ceps_remover = ['95670000', '36800000', '12120000', '41603270', '60000000', '74030000']
df = df[~df['Cep'].isin(ceps_remover)]

#### Retornando Estado, Cidade e Bairro referente ao CEP

In [167]:
import requests
import time

sigla_para_estado = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
    'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo',
    'GO': 'Goiás', 'MA': 'Maranhão', 'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba', 'PR': 'Paraná',
    'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}

# df['Cidade'] = ''
# df['Estado'] = ''
# df['Bairro'] = ''
# df['É São Paulo?'] = ''
# df['Zona'] = ''

for i, cep in enumerate(df['Cep']):
    try:
        response = requests.get(f"https://viacep.com.br/ws/{cep}/json/") #viacep
        # response = requests.get(f"https://cep.awesomeapi.com.br/json/{cep}") # AwesomeAPI

        data = response.json()

        if 'erro' not in data:
            # AwesomeAPI
            # df.at[i, 'Cidade'] = data.get('city', '')
            # sigla_uf = data.get('state', '')
            # df.at[i, 'Estado'] = sigla_para_estado.get(sigla_uf, sigla_uf)

            # bairro = data.get('district')
            # df.at[i, 'Bairro'] = bairro if bairro else 'Não informado'

            # ViaCep
            df.at[i, 'Cidade'] = data.get('localidade', '')
            df.at[i, 'Estado'] = data.get('estado', '')

            bairro = data.get('bairro')
            df.at[i, 'Bairro'] = bairro if bairro else 'Não informado'

        else:
            print(f"CEP inválido: {cep}")
        
        time.sleep(0.2)

    except Exception as e:
        print(f"Erro no CEP {cep}: {e}")


#### Movendo Colunas relevantes para novo excel

In [60]:
colunas_desejadas = ['Cep', 'Estado', 'Cidade', 'Bairro', 'É São Paulo?', 'Zona', 'Número de sacos de lixo', 'Litragem dos sacos', 'Peso total dos resíduos', 'Quantidade total de itens coletados']
df = df[colunas_desejadas]

#### Gerando Coluna de True ou False para São Paulo

In [175]:
df['É São Paulo?'] = df['Cidade'].str.strip().str.lower().apply(lambda x: 'Sim' if x == 'são paulo' else 'Não')

#### Arrumando valores nulos e padronizando tipos de dados

In [62]:
import pandas as pd
import numpy as np
import re

def extrair_inteiro(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if 'não informado' in valor or not re.search(r'\d+', valor):
        return np.nan
    numeros = re.findall(r'\d+', valor)
    return int(numeros[0]) if numeros else np.nan

def extrair_float(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if 'não informado' in valor or not re.search(r'\d', valor):
        return np.nan
    return np.nan

df['Litragem dos sacos'] = df['Litragem dos sacos'].apply(extrair_inteiro)
df['Número de sacos de lixo'] = df['Número de sacos de lixo'].apply(extrair_inteiro)
df['Quantidade total de itens coletados'] = df['Quantidade total de itens coletados'].apply(extrair_inteiro)
df['Peso total dos resíduos'] = df['Peso total dos resíduos'].apply(extrair_float)


#### Gerando Coluna para Zona de São Paulo

In [177]:
zonas = {
    "Centro": [
        "Sé", "República", "Bom Retiro", "Barra Funda",
        "Santa Cecília", "Consolação", "Bela Vista",
        "Liberdade", "Cambuci"
    ],
    "Zona Norte": [
        "Santana", "Vila Maria", "Tucuruvi", "Jaçanã", "Tremembé",
        "Mandaqui", "Casa Verde", "Cachoeirinha", "Limão",
        "Brasilândia", "Freguesia do Ó", "Pirituba",
        "São Domingos", "Jaraguá", "Perus", "Horto Florestal"
    ],
    "Zona Leste": [
        "Belém", "Brás", "Pari", "Mooca", "Vila Prudente", "Água Rasa",
        "São Lucas", "Sapopemba", "Tatuapé", "Vila Formosa", "Vila Carrão",
        "Vila Matilde", "Artur Alvim", "Penha", "Cangaiba", "Ponte Rasa",
        "Ermelino Matarazzo", "São Mateus", "São Miguel",
        "Itaim Paulista", "Itaquera", "Guaianazes", "Jardim Santo André", "Cidade Patriarca", "Vila Gomes Cardim"
    ],
    "Zona Sul": [
        "Vila Mariana", "Moema", "Saúde", "Ipiranga (Vila Monumento)",
        "Vila Mariana (Jardim da Glória)", "Ipiranga", "Sacomã", "Jabaquara",
        "Cidade Ademar", "Pedreira", "Itaim Bibi", "Campo Belo",
        "Santo Amaro", "Cidade Dutra", "Grajaú", "Parelheiros",
        "Capela do Socorro", "Campo Limpo", "Capão Redondo","Jardim Celeste", "Vila Monte Alegre", "Vila Clara", "Parque do Lago", "Jardim Catanduva", "Jardim Figueira Grande", "Chácara da Enseada", "Cidade Ipava", "Vila Santa Lúcia", "Morro do Índio", "Mirandópolis"
    ],
    "Zona Oeste": [
        "Jardim Paulista", "Lapa", "Perdizes", "Jaguara",
        "Vila Leopoldina", "Jaguaré", "Rio Pequeno",
        "Pinheiros", "Alto de Pinheiros", "Butantã", "Morumbi", "Vila Sônia", "Cidade Monções"
    ]
}

def buscar_zona(row):
    cidade = str(row['Cidade']).strip().lower()
    bairro = str(row['Bairro']).strip().lower()

    if cidade != 'são paulo' or not bairro:
        return ''

    for nome_zona, bairros in zonas.items():
        for b in bairros:
            if b.lower() in bairro:
                return nome_zona
    return 'Desconhecida'

df['Zona'] = df.apply(buscar_zona, axis=1)


#### Gerando Excel

In [158]:
df.to_excel('limpaBrasilOrganizado.xlsx', index=False) 

In [211]:
df.head(50)

,Cep,Estado,Cidade,Bairro,É São Paulo?,Zona,Latitude,Longitude
0,08022120,São Paulo,São Paulo,Vila Clara,Sim,Zona Sul,-23.495161,-46.418803
1,08390355,São Paulo,São Paulo,Jardim Santo André,Sim,Zona Leste,-23.632526,-46.444597
2,04052020,São Paulo,São Paulo,Mirandópolis,Sim,Zona Sul,-23.610538,-46.639605
3,04366001,São Paulo,São Paulo,Cidade Ademar,Sim,Zona Sul,-23.668394,-46.651485
4,04366001,São Paulo,São Paulo,Cidade Ademar,Sim,Zona Sul,-23.668394,-46.651485
6,04564003,São Paulo,São Paulo,Cidade Monções,Sim,Zona Oeste,-23.612145,-46.685977
8,05873270,São Paulo,São Paulo,Morro do Índio,Sim,Zona Sul,-23.690326,-46.777943
9,04940010,São Paulo,São Paulo,Vila Santa Lúcia,Sim,Zona Sul,-23.690015,-46.767542
10,05767320,São Paulo,São Paulo,Jardim Catanduva,Sim,Zona Sul,-23.633076,-46.761499
11,05767320,São Paulo,São Paulo,Jardim Catanduva,Sim,Zona Sul,-23.633076,-46.761499


In [66]:
# Não foi rodado ainda!

# Resolvendo Problema

In [200]:
df = pd.read_excel('limpaBrasil_.xlsx')

In [201]:
df['Cep'] = pd.to_numeric(df['Cep'], errors='coerce').astype('Int64')

In [205]:
df = df.dropna(subset=['Cep'])

In [206]:
df['Cep'] = df['Cep'].astype(str)

In [207]:
df['Cep'] = df['Cep'].astype(str).str.zfill(8)

df = df[df['Cep'].str.len() == 8]

In [212]:
df.to_excel('limpaBrasil_sp.xlsx', index=False) 

In [204]:
df['Cep'] = df['Cep'].replace(
    to_replace=["0000<NA>", "<NA>", "nan", "", "None"], 
    value=np.nan
)

In [208]:
df = df.drop(columns=["Quantidade total de itens coletados", "Peso total dos resíduos", "Litragem dos sacos", "Número de sacos de lixo"])

# APAGAR DPS

In [210]:
import requests
import pandas as pd

# Função para buscar lat/lon diretamente da AwesomeAPI
def get_lat_lon_from_awesomeapi(cep):
    try:
        response = requests.get(f"https://cep.awesomeapi.com.br/json/{cep}")
        if response.status_code == 200:
            data = response.json()
            lat = float(data['lat'])
            lon = float(data['lng'])
            return pd.Series([lat, lon])
    except Exception as e:
        print(f"Erro para o CEP {cep}: {e}")
    return pd.Series([None, None])

# Garantir que o CEP esteja como string de 8 dígitos
df['Cep'] = df['Cep'].astype(str).str.zfill(8)

# Aplicar a função ao DataFrame
df[['Latitude', 'Longitude']] = df['Cep'].apply(get_lat_lon_from_awesomeapi)

# Mostrar resultado
print(df[['Cep', 'Latitude', 'Longitude']].head())


        Cep   Latitude  Longitude
0  08022120 -23.495161 -46.418803
1  08390355 -23.632526 -46.444597
2  04052020 -23.610538 -46.639605
3  04366001 -23.668394 -46.651485
4  04366001 -23.668394 -46.651485
